In [50]:
import joblib
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer

loaded_model = joblib.load('sentiment_analysis_model.pkl')
loaded_vect = joblib.load('count_vectorizer.pkl')
loaded_tfidf = joblib.load('tfidf_transformer.pkl')

In [51]:
import re
def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result
def remove_num(texts):
   output = re.sub(r'\d+', '', texts)
   return output
def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)
def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string 
def remove_symbols(x):
    cleaned_string = re.sub(r"[^a-zA-Z0-9?!.,;:!]+", ' ', x)
    return cleaned_string
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',','))
    return final

stop=set(stopwords.words("english"))
stemmer=PorterStemmer()
lemma=WordNetLemmatizer()

def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
def Stemming(text):
   stem=[]
   stopword = stopwords.words('english')
   snowball_stemmer = SnowballStemmer('english')
   word_tokens = nltk.word_tokenize(text)
   stemmed_word = [snowball_stemmer.stem(word) for word in word_tokens]
   stem=' '.join(stemmed_word)
   return stem

def cleaning(df, review_col):
    df[review_col] = df[review_col].apply(clean)
    df[review_col] = df[review_col].apply(deEmojify)
    df[review_col] = df[review_col].str.lower()
    df[review_col] = df[review_col].apply(remove_num)
    df[review_col] = df[review_col].apply(remove_symbols)
    df[review_col] = df[review_col].apply(remove_punctuation)
    df[review_col] = df[review_col].apply(remove_stopword)
    df[review_col] = df[review_col].apply(unify_whitespaces)
    df[review_col] = df[review_col].apply(Stemming)
    return df


In [60]:
def predict_sentiment(review_text):
    # Preprocess the review text
    df = pd.DataFrame({'review':[review_text]})
    df = cleaning(df, 'review')
    review_text = df['review'].iloc[0]  # Extract preprocessed review text
    
    # Vectorize and transform the preprocessed text
    text_vectorized = loaded_vect.transform([review_text])
    text_tfidf = loaded_tfidf.transform(text_vectorized)

    # Predict sentiment
    sentiment = loaded_model.predict(text_tfidf)

    return "Positive" if sentiment[0] == 1 else "Negative"

In [62]:
sample_review = "This game is amazing! I love the graphics and gameplay."
print("Sample Review:", sample_review)
print("Predicted Sentiment:", predict_sentiment(sample_review))

Sample Review: This game is amazing! I love the graphics and gameplay.
Predicted Sentiment: Positive


In [63]:
sample_review2 = "This was a very disappointing game"
print("\nSample Review:", sample_review2)
print("Predicted Sentiment:", predict_sentiment(sample_review2))


Sample Review: This was a very disappointing game
Predicted Sentiment: Negative
